In [ ]:
import datetime
import matplotlib
import matplotlib.pyplot as plt
import os
import random
import io
import shutil
# import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage
import tensorflow as tf
import pathlib
import itertools
import random

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

from object_detection.utils import config_util
from object_detection.builders import model_builder
%matplotlib inline



In [ ]:
current_datetime=datetime.datetime.now()
print(f"Calf was last happy on: {current_datetime.strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# import sys 
# import os
# print(f"Current working directory:{os.getcwd()}")
# if not os.path.exists("models"):
#     !git clone --quiet https://github.com/tensorflow/models.git # clone the tensorflow models repository
# else:
#     print("Tensorflow models already cloned")

# import subprocess

# try:
#     cmd = 'cd /home/woody/iwfa/iwfa030h/Calf_Detection/models/research && python -c "import object_detection; print(object_detection.__path__)"'
#     result = subprocess.check_output(cmd, shell=True, stderr=subprocess.STDOUT)
#     module_installed = True
# except subprocess.CalledProcessError:
#     module_installed = False

# if not module_installed:
#     print("object_detection module is not installed.")
#     !cd models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install .

# else:
#     print("object_detection module is already installed.")

# %matplotlib inline
# Download EfficientDet model 
# # TODO: 
# # 1. Configure the directory path for the model download.
# import wget
# model_link = "http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz"
# model_tar_name="efficientdet_d0_coco17_tpu-32.tar.gz"

# # Check if model is already downloadeds
# if not os.path.exists(model_tar_name):
#     wget.download(model_link)
#     # Unzip
#     !tar -zxvf {model_tar_name} 
# else:
#     print(f"Model {model_tar_name} already downloaded")


In [ ]:
!nvidia-smi
print(tf.config.list_physical_devices('GPU'))
print(tf.keras.backend.clear_session())
!nvidia-smi
# !kill 418413



In [ ]:
# Emptying the previous log files.


# #clearing session memory
# print(tf.keras.backend.clear_session())

# def empty_directory(directory_path):
#     for filename in os.listdir(directory_path):
#         file_path = os.path.join(directory_path, filename)
#         try:
#             if os.path.isfile(file_path) or os.path.islink(file_path):
#                 os.unlink(file_path)
#             elif os.path.isdir(file_path):
#                 shutil.rmtree(file_path)
#         except Exception as e:
#             print(f"Failed to delete {file_path}. Reason: {e}")

# #  Emptying the previous training_files
# dir_path_1 = "Calf_Detection/new_app/training_ed0"
# empty_directory(dir_path_1)

# #  Emptying the previous finetuned_files
# dir_path_2 = "Calf_Detection/new_app/finetuned_ed0"
# empty_directory(dir_path_2)

# # Training the model
# !python3 Calf_Detection/models/research/object_detection/model_main_tf2.py \
#     --pipeline_config_path=Calf_Detection/new_app/config/efficientdet_d0_coco17_tpu-32.config \
#     --model_dir=Calf_Detection/new_app/training_ed0 \
#     --alsologtostderr

In [ ]:
# Visualization in tensorboard 
# Todo:
# 1. Check Tensorboard visualization
# !cd Calf_Detection/new_app/training_ed0 && tensorboard --logdir=.
# %load_ext tensorboard
# !kill 411862
# %reload_ext tensorboard
# %tensorboard --logdir=.

In [ ]:
# Export the model
pipeline_file = 'Calf_Detection/new_app/config/efficientdet_d0_coco17_tpu-32.config'
last_model_path = 'Calf_Detection/new_app/training_ed0'
output_directory = "Calf_Detection/new_app/finetuned_ed0"
# !python3 Calf_Detection/models/research/object_detection/exporter_main_v2.py \
#     --trained_checkpoint_dir {last_model_path} \
#     --output_directory {output_directory} \
#     --pipeline_config_path {pipeline_file}

In [ ]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.
  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.
  Args:
    path: a file path.
  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def plot_detections(image_np,
                    boxes,
                    classes,
                    scores,
                    category_index,
                    figsize=(12, 16),
                    image_name=None):
  image_np_with_annotations = image_np.copy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_annotations,
      boxes,
      classes,
      scores,
      category_index,
      use_normalized_coordinates=True,
      min_score_thresh=0.8)
  if image_name:
    plt.imsave(image_name, image_np_with_annotations)
  else:
    plt.imshow(image_np_with_annotations)

In [ ]:
filenames = list(pathlib.Path('Calf_Detection/new_app/training_ed0').glob('*.index'))
filenames.sort()

# recover our saved model
model_dir = 'Calf_Detection/new_app/training_ed0'
# Adding the last checkpoint
configs = config_util.get_configs_from_pipeline_file(pipeline_file)
model_config = configs['model']
detection_model = model_builder.build(
      model_config=model_config, is_training=False)

print(detection_model)
# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(
      model=detection_model)
ckpt.restore(os.path.join(str(filenames[-1]).replace('.index',''))).expect_partial()
print(os.path.join(str(filenames[-1]).replace('.index','')))

def get_model_detection_function(model):
  """Get a tf.function for detection."""

  @tf.function
  def detect_fn(image):
    """Detect objects in image."""

    image, shapes = model.preprocess(image)
    prediction_dict = model.predict(image, shapes)
    detections = model.postprocess(prediction_dict, shapes)

    return detections, prediction_dict, tf.reshape(shapes, [-1])

  return detect_fn

detect_fn = get_model_detection_function(detection_model)

In [ ]:
# map labels for inference decoding
label_map_path = configs['eval_input_config'].label_map_path
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(
    label_map,
    max_num_classes=label_map_util.get_max_label_map_index(label_map),
    use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map_dict = label_map_util.get_label_map_dict(label_map, use_display_name=True)

In [ ]:
#run detector on test image
#it takes a little longer on the first run and then runs at normal speed. 
# Calf_Detection/new_data/annotated_label_img_training_data/2023-10-12_09-32-01.jpg
# TEST_IMAGE_PATHS = glob.glob('Calf_Detection/new_app/test/2019-06-29_090001.jpg')
TEST_IMAGE_PATHS = glob.glob('Calf_Detection/new_data/annotated_label_img_training_data/2023-10-12_09-32-01.jpg')
image_path = random.choice(TEST_IMAGE_PATHS)
image_np = load_image_into_numpy_array(image_path)

input_tensor = tf.convert_to_tensor(
    np.expand_dims(image_np, 0), dtype=tf.float32)
detections, predictions_dict, shapes = detect_fn(input_tensor)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections,
      detections['detection_boxes'][0].numpy(),
      (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
      detections['detection_scores'][0].numpy(),
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=1,
      min_score_thresh=.1,
      agnostic_mode=False,
)

plt.figure(figsize=(12,16))
plt.imshow(image_np_with_detections)
plt.axis("off")
plt.show()

In [ ]:
def detect_and_visualize_images(directory_path, detect_fn, category_index, num_images=10):
    """
    Detect and visualize objects in random images from a directory.

    Parameters:
    directory_path (str): Path to the directory containing images.
    detect_fn (function): Detection function.
    category_index (dict): Dictionary for category indexing.
    num_images (int): Number of images to process.
    """
    # Get all image paths from the directory
    all_image_paths = glob.glob(f'{directory_path}/*.jpg')
    if len(all_image_paths) < num_images:
        num_images = len(all_image_paths)
        print(f"Only {num_images} images found in the directory.")

    # Select random images
    selected_image_paths = random.sample(all_image_paths, num_images)

    # Set up plot
    fig, axes = plt.subplots(nrows=num_images, ncols=1, figsize=(12, num_images * 16))
    if num_images == 1:
        axes = [axes]

    for ax, image_path in zip(axes, selected_image_paths):
        # Load and detect objects in the image
        image_np = load_image_into_numpy_array(image_path)
        input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
        detections, predictions_dict, shapes = detect_fn(input_tensor)

        # Visualize detections
        label_id_offset = 1
        image_np_with_detections = image_np.copy()

        viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'][0].numpy(),
            (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
            detections['detection_scores'][0].numpy(),
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=1,
            min_score_thresh=.1,
            agnostic_mode=False,
        )

        # Plot the image
        ax.imshow(image_np_with_detections)
        ax.axis('off')

    plt.tight_layout()
    plt.show()

detect_and_visualize_images('Calf_Detection/new_data/pairs_annotated_label_img_training_data', detect_fn, category_index)


In [ ]:
import time

sample_image_path = 'Calf_Detection/new_app/test/2019-06-29_090001.jpg'
sample_image_np = load_image_into_numpy_array(sample_image_path)

# Define the number of inference iterations

# Assuming 24 fps x 60 seconds = 1440 frames per minute
num_iterations = 1440

total_inference_time = 0

for _ in range(num_iterations):
    input_tensor = tf.convert_to_tensor(
        np.expand_dims(sample_image_np, 0), dtype=tf.float32)

    start_time = time.time()
    detections, predictions_dict, shapes = detect_fn(input_tensor)
    end_time = time.time()
    
    inference_time = end_time - start_time
    total_inference_time += inference_time

average_inference_time = total_inference_time 
print(f"Total Inference Time for 1 Minute: {average_inference_time:.5f} seconds")


In [ ]:
# # Evaluate the model
# !python3 ./models/research/object_detection/model_main_tf2.py \
#     --pipeline_config_path=new_app/config/efficientdet_d0_coco17_tpu-32.config \
#     --model_dir=new_app/training_ed0 \
#     --checkpoint_dir=new_app/training_ed0 \
#     --alsologtostderr
